# 기본 필요 정보
- 런타임 재시동시 기본으로 실행해야할 것들.

In [48]:
# 필요 라이브러리 import 
import numpy as np
import re
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
# mount

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 사용한 데이터셋
- AI-Hub `소상공인 고객 주문 질의-응답 텍스트` ([링크 ](https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=102))
- '카페' 업종 데이터 활용.

# 데이터 불러오기 / 전처리

### 데이터 불러오기

In [12]:
df = pd.read_csv('/content/drive/MyDrive/카페_train.csv')
df.head()

,IDX,발화자,발화문,카테고리,QA번호,QA여부,감성,인텐트,가격,수량,크기,장소,조직,사람,시간,날짜,상품명,상담번호,상담내순번
0,1,s,고객님 혹시 무슨 문제 있으신가요?,카페,5,q,m,제품_불량_질문,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9252,9
1,2,c,네 이상해요,카페,5,a,m,제품_불량_질문,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9252,10
2,3,c,빙수 위에 뿌려진 과일에서 쉰 냄새가 나는데 어떻게 하면 되죠?,카페,20591,q,n,제품_불량_질문,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,빙수|과일,20591,1
3,4,s,"죄송합니다, 폐기처분 하고 새 빙수를 제조해드리겠습니다.",카페,20591,a,m,제품_불량_질문,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,빙수,20591,2
4,5,c,마카롱 주문했는데 마카롱이 엄청 딱딱해요.,카페,66996,q,n,제품_불량_질문,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,마카롱,66996,1


In [13]:
# dtype check
df.info() # 전체 데이터 169632개. null값 확인 필요.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 169633 entries, 0 to 169632
Data columns (total 19 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   IDX     169633 non-null  int64 
 1   발화자     169633 non-null  object
 2   발화문     169633 non-null  object
 3   카테고리    169633 non-null  object
 4   QA번호    169633 non-null  int64 
 5   QA여부    169633 non-null  object
 6   감성      169633 non-null  object
 7   인텐트     169633 non-null  object
 8   가격      9716 non-null    object
 9   수량      7064 non-null    object
 10  크기      794 non-null     object
 11  장소      2620 non-null    object
 12  조직      1105 non-null    object
 13  사람      429 non-null     object
 14  시간      1871 non-null    object
 15  날짜      3262 non-null    object
 16  상품명     85176 non-null   object
 17  상담번호    169633 non-null  int64 
 18  상담내순번   169633 non-null  int64 
dtypes: int64(4), object(15)
memory usage: 24.6+ MB


In [15]:
# null 확인
print('count null \n', df.isnull().sum())

count null 
 IDX           0
발화자           0
발화문           0
카테고리          0
QA번호          0
QA여부          0
감성            0
인텐트           0
가격       159917
수량       162569
크기       168839
장소       167013
조직       168528
사람       169204
시간       167762
날짜       166371
상품명       84457
상담번호          0
상담내순번         0
dtype: int64


In [17]:
# null 값 포함 칼럼은 제거하고 사용
features = ['발화자', '발화문', 'QA번호', 'QA여부', '감성', '인텐트', '상담번호', '상담내순번']

df = df[features]

df.head()

,발화자,발화문,QA번호,QA여부,감성,인텐트,상담번호,상담내순번
0,s,고객님 혹시 무슨 문제 있으신가요?,5,q,m,제품_불량_질문,9252,9
1,c,네 이상해요,5,a,m,제품_불량_질문,9252,10
2,c,빙수 위에 뿌려진 과일에서 쉰 냄새가 나는데 어떻게 하면 되죠?,20591,q,n,제품_불량_질문,20591,1
3,s,"죄송합니다, 폐기처분 하고 새 빙수를 제조해드리겠습니다.",20591,a,m,제품_불량_질문,20591,2
4,c,마카롱 주문했는데 마카롱이 엄청 딱딱해요.,66996,q,n,제품_불량_질문,66996,1


In [18]:
# null 재확인
print('count null \n', df.isnull().sum()) #=> checked.

count null 
 발화자      0
발화문      0
QA번호     0
QA여부     0
감성       0
인텐트      0
상담번호     0
상담내순번    0
dtype: int64


In [19]:
# 인텐트 unique 확인해보기
print(df['인텐트'].unique()) #=> 너무 많아서 활용이 어려울 것 같긴 하지만, 어떻게 쓸 수 있을지는 두고보자. 

['제품_불량_질문' '멤버십_오류_요청' '배송_방법_요청' 'AS_일반_질문' '제품_소재_요청' '포장_비용_질문'
 '행사_날짜_확인' '제품_재고_확인' '주문_변경_확인' '결제_일반_질문' '주문_추가_질문' '멤버십_오류_질문'
 '멤버십_사용_요청' '제품_추천_확인' '멤버십_가입_확인' '매장_정보_확인' '제품_불량_요청' '예약_취소_질문'
 '제품_날짜_요청' '제품_불량_확인' '예약_방법_확인' '주문_취소_요청' '제품_날짜_확인' '결제_추가_질문'
 '배송_일반_질문' '제품_소재_비교' '제품_원산지_확인' '예약_취소_요청' '결제_할인_질문' '행사_날짜_요청'
 '행사_유형_질문' '예약_방법_질문' '결제_재결제_요청' '행사_일반_질문' '매장_부대시설_확인' '결제_수단_질문'
 '교환|반품|환불_비용_질문' '행사_정보_요청' '제품_가격_질문' '결제_방식_질문' '주문_비품_요청' '행사_기간_질문'
 '예약_방법_요청' '배송_날짜_질문' '제품_방법_요청' '결제_재결제_확인' '결제_일반_확인' '매장_이용_질문'
 '결제_오류_확인' '멤버십_적립_확인' '결제_추가_확인' '멤버십_적립_질문' '멤버십_일반_요청' '주문_제품_요청'
 '제품_정보_질문' '행사_일반_요청' '결제_오류_요청' '결제_할인_확인' '제품_구성_질문' '제품_날짜_질문'
 '주문_오류_질문' '제품_방법_질문' '포장_방식_확인' '매장_이용_확인' '매장_정보_질문' '교환|반품|환불_시간_확인'
 '제품_추천_질문' '예약_비용_질문' '배송_날짜_요청' '주문_추가_요청' '주문_추가_확인' '웹사이트_가입_질문'
 '배송_지역_요청' '결제_재결제_질문' '멤버십_일반_확인' '교환|반품|환불_시간_요청' '제품_입고_요청'
 '멤버십_오류_확인' '주문_변경_요청' 'AS_일반_요청' '제품_품질_확인' '배송_오류_질문' '멤버십_가입_질문'
 '매장_부대시설_요청' '제품_재고_질문' '제품_용도_질문' '

### 데이터 전처리

**바꾸고 싶은 데이터 형태**
- 질문과 그에 맞는 답변이 Q, A 칼럼에 매칭되어 한 행에 저장되어 있는 형태가 최종적으로 원하는 결과.
- 원본 데이터의 발화문을 Q-A가 연속되는 행으로 정렬할 수 있다면 for loop를 돌려서 매칭해 저장할 수 있지 않을까?
- 일단 원본 데이터의 다양한 칼럼들을 이용해 소팅하며 규칙을 찾아보자.


In [24]:
# 하나의 QA번호당 평균적으로 몇 건의 대화를 나누는지 한 번 보기
print(df.groupby("QA번호")['발화자'].count().mean())
print(df.groupby("QA번호")['발화문'].count().max()) # 2946건? 'QA번호' 칼럼에 노이즈가 있을 수도 있겠다. 활용에 주의하기.
df.groupby("QA번호")['발화문'].count().min()

2.9189193839800396
2946


1

In [26]:
# 가장 기본인 '상담번호' 기준으로 정렬하기
df = df.sort_values('상담번호')
df.head(10)

,발화자,발화문,QA번호,QA여부,감성,인텐트,상담번호,상담내순번
86168,s,네 알겠습니다.,14,a,m,주문_추가_요청,1,7
1236,s,"네, 가능합니다.",4,a,m,배송_방법_요청,1,2
115932,s,네.,6,a,m,주문_제품_질문,1,3
31368,c,오메기떡 1박스가 얼마죠?,1,q,m,제품_가격_질문,1,1
6219,c,"#이름#, #전화번호#, #주소# 2로입니다.",8,a,m,멤버십_가입_확인,1,4
6238,s,"성함과 연락처, 주소를 말씀해주세요.",7,q,m,멤버십_가입_확인,1,4
130484,s,3만원 미만 구매 시 택배비 3천원입니다.,12,a,m,배송_비용_질문,1,6
2432,c,택배로 받을 수 있어요?,3,q,m,배송_방법_요청,1,2
115125,c,지금 바로 주문할 수 있나요?,5,q,m,주문_제품_질문,1,3
42804,c,아메리카노 시원하게 한 잔만 주실래요?,1,q,m,주문_제품_요청,1,1


In [27]:
# 기준 추가 ('상담내순번' 칼럼도 이용해봤는데 정렬이 더 흐트러졌음. 해당 칼럼은 기존 라벨링을 신뢰할 수 없는 것으로 간주)
df = df.sort_values(['상담번호', '인텐트', 'QA번호'])
df.head(40)

,발화자,발화문,QA번호,QA여부,감성,인텐트,상담번호,상담내순번
6238,s,"성함과 연락처, 주소를 말씀해주세요.",7,q,m,멤버십_가입_확인,1,4
6219,c,"#이름#, #전화번호#, #주소# 2로입니다.",8,a,m,멤버십_가입_확인,1,4
37342,c,언제쯤 받을 수 있어요?,9,q,m,배송_날짜_질문,1,5
2432,c,택배로 받을 수 있어요?,3,q,m,배송_방법_요청,1,2
1236,s,"네, 가능합니다.",4,a,m,배송_방법_요청,1,2
130484,s,3만원 미만 구매 시 택배비 3천원입니다.,12,a,m,배송_비용_질문,1,6
31368,c,오메기떡 1박스가 얼마죠?,1,q,m,제품_가격_질문,1,1
33077,s,2만원입니다.,2,a,m,제품_가격_질문,1,1
42804,c,아메리카노 시원하게 한 잔만 주실래요?,1,q,m,주문_제품_요청,1,1
42805,s,"네, 잠시만 기다려주시면 바로 준비해서 드리도록 하겠습니다.",1,a,m,주문_제품_요청,1,2


**'QA여부' 칼럼 기준으로 q 하나 뽑고 a 하나 뽑고, 그 다음 q 뽑고 a 하나 뽑고 하는 식으로 'Q, A' 데이터프레임 만들어서 전반적으로 한 번 살펴보자.** 

In [28]:
# QA여부 = q (질문)
df_q = df[(df['QA여부']=='q')]

# QA여부 = a (답변)
df_a = df[(df['QA여부']=='a')]

In [29]:
# 조금만 빼서 질답 비교 테스트
test_q = df_q.head(50)
test_a = df_a.head(50)

qlist = []
alist = []

for q, a in zip(test_q['발화문'], test_a['발화문']):
  qlist.append(q)
  alist.append(a)

test_qa = pd.DataFrame(columns = ['Q', 'A'])

test_qa['Q'] = qlist
test_qa['A'] = alist

test_qa # 대화가 어긋나니 상담번호별 총 대화 수가 홀수개인 것들 날려보자.

,Q,A
0,"성함과 연락처, 주소를 말씀해주세요.","#이름#, #전화번호#, #주소# 2로입니다."
1,언제쯤 받을 수 있어요?,"네, 가능합니다."
2,택배로 받을 수 있어요?,3만원 미만 구매 시 택배비 3천원입니다.
3,오메기떡 1박스가 얼마죠?,2만원입니다.
4,아메리카노 시원하게 한 잔만 주실래요?,"네, 잠시만 기다려주시면 바로 준비해서 드리도록 하겠습니다."
5,지금 바로 주문할 수 있나요?,네.
6,멤버십 가입은 어떻게 해요?,네 알겠습니다.
7,혜택 같은 거 있어요?,저희 매장 어플리케이션에서 하실 수 있어요.
8,신규 가입 쿠폰은 뭐 주시나요?,저희 매장 이름과 동일합니다.
9,쿠폰 찍을 수 있어요?,"신규 가입 쿠폰, 생일 축하 쿠폰, 멤버십 한정 깜짝 이벤트 쿠폰이 있습니다."


In [31]:
## 상담번호별 총 대화 수가 홀수개인 것들 날려보자.

# 상담번호 별 발화문 수 확인
a = df.groupby('상담번호').count()['인텐트']
a

상담번호
1        13
2        19
3        17
4         8
5         9
         ..
75846     2
75847     2
75849     2
75851     2
75852     2
Name: 인텐트, Length: 58586, dtype: int64

In [33]:
# 데이터 프레임으로 저장
df_count = pd.DataFrame(columns = ['상담번호', '카운트'])

df_count['상담번호'] = list(a.index)
df_count['카운트'] = list(a.values)

df_count.head() #=> double-checked

,상담번호,카운트
0,1,13
1,2,19
2,3,17
3,4,8
4,5,9


In [34]:
# 홀수 개 가진 상담번호 찾기

councel_num = []

for n, c in zip(df_count['상담번호'], df_count['카운트']):
  if c % 2 != 0:
    councel_num.append(n)
  else:
    pass

print(len(councel_num), '개\n') # 967개나 되긴 해도 최종 활용 데이터셋 숫자만 충분하다면 제거해도 문제 안될 것.

967 개



In [35]:
# 해당 상담번호 제거
df_new = df[(~df['상담번호'].isin(councel_num))]

df_new

,발화자,발화문,QA번호,QA여부,감성,인텐트,상담번호,상담내순번
168056,c,여기 화장실은 어디에 있죠?,7,q,m,매장_부대시설_질문,4,4
168788,c,화장실 비밀번호 있나요?,9,q,m,매장_부대시설_질문,4,5
168980,s,0000# 누르시면 돼요.,10,a,m,매장_부대시설_질문,4,5
14508,s,"네, 화장실에 데리고 가셔도 됩니다.",14,a,m,매장_부대시설_확인,4,7
105263,s,"네, 같이 내어드릴게요.",6,a,m,매장_이용_요청,4,3
...,...,...,...,...,...,...,...,...
80513,c,혹시 버터핑거 팬케익 오늘 영업하시나요?,75849,q,m,매장_정보_질문,75849,1
108528,c,11시까지 버터핑거 팬케익 하나랑 칠리버거 하나 포장해주시겠어요?,75851,q,m,포장_방식_요청,75851,1
108529,s,11시까지 버터핑거 팬케익 하나랑 칠리버거 하나 포장해드리습니다.,75851,a,m,포장_방식_요청,75851,2
64275,c,오메기떡 종류에는 뭐가 있나요?,75852,q,m,제품_정보_질문,75852,1


In [36]:
df_new.shape 

(149692, 8)

In [37]:
len(list(df_new['상담번호'].unique())) # 제거 후에도 데이터 이정도면 충분해 보인다.

57619

In [ ]:
# 전반적으로 한 번 살펴보기
df_new.head(50) # 근데 짝수개라고 꼭 정렬이 맞는 것도 아니다. 다른 방법을 찾아야 한다. 

,발화자,발화문,QA번호,QA여부,감성,인텐트,상담번호,상담내순번
168056,c,여기 화장실은 어디에 있죠?,7,q,m,매장_부대시설_질문,4,4
168788,c,화장실 비밀번호 있나요?,9,q,m,매장_부대시설_질문,4,5
168980,s,0000# 누르시면 돼요.,10,a,m,매장_부대시설_질문,4,5
14508,s,"네, 화장실에 데리고 가셔도 됩니다.",14,a,m,매장_부대시설_확인,4,7
105263,s,"네, 같이 내어드릴게요.",6,a,m,매장_이용_요청,4,3
38439,s,죄송하지만 혹시 모를 사고 예방 차원차 반려동물은 맡아드리지 않습니다.,12,a,m,매장_이용_질문,4,6
79125,c,강아지 들어가도 되는 매장 맞죠?,1,q,m,매장_이용_확인,4,1
79059,s,"네, 같이 들어오셔도 돼요.",2,a,m,매장_이용_확인,4,1
168592,s,"아니요, 화장실 옆에 흡연실 위치하고 있습니다.",10,a,m,매장_부대시설_질문,7,5
105504,c,너무 추워서 그런데 히터 좀 틀어주세요.,1,q,n,매장_이용_요청,7,1


위와 같이 하면 어떻게 해도 Q, A를 제대로 매칭할 수 있는 규칙이 나오지 않는다.<br>
- **상담번호 총 카운트가 2개인 것만 사용하도록 하자. 결과 데이터만 충분하면 우선 괜찮을 것 같고, 데이터 수집 방식에 대한 내용은 필요시 추후 개선사항으로 넣기**

In [38]:
# 다시 원래 데이터프레임 사용해서 재처리하자.
df

,발화자,발화문,QA번호,QA여부,감성,인텐트,상담번호,상담내순번
6238,s,"성함과 연락처, 주소를 말씀해주세요.",7,q,m,멤버십_가입_확인,1,4
6219,c,"#이름#, #전화번호#, #주소# 2로입니다.",8,a,m,멤버십_가입_확인,1,4
37342,c,언제쯤 받을 수 있어요?,9,q,m,배송_날짜_질문,1,5
2432,c,택배로 받을 수 있어요?,3,q,m,배송_방법_요청,1,2
1236,s,"네, 가능합니다.",4,a,m,배송_방법_요청,1,2
...,...,...,...,...,...,...,...,...
80513,c,혹시 버터핑거 팬케익 오늘 영업하시나요?,75849,q,m,매장_정보_질문,75849,1
108528,c,11시까지 버터핑거 팬케익 하나랑 칠리버거 하나 포장해주시겠어요?,75851,q,m,포장_방식_요청,75851,1
108529,s,11시까지 버터핑거 팬케익 하나랑 칠리버거 하나 포장해드리습니다.,75851,a,m,포장_방식_요청,75851,2
64275,c,오메기떡 종류에는 뭐가 있나요?,75852,q,m,제품_정보_질문,75852,1


In [40]:
df_count.head()

,상담번호,카운트
0,1,13
1,2,19
2,3,17
3,4,8
4,5,9


In [41]:
# 2개의 대화만 가진 상담번호 저장하기

councel_num = []

for n, c in zip(df_count['상담번호'], df_count['카운트']):
  if c == 2:
    councel_num.append(n)
  else:
    pass

print(len(councel_num), '개\n') # 샘플 수 56,163개 확인. 충분해 보인다. 

56163 개



In [42]:
# 해당 상담번호만 남기기
df_new = df[(df['상담번호'].isin(councel_num))]

df_new.head(10) #=> 일단 좋아 보인다. 근데 생각해보면 당연히 그럴 수 밖에 없는 듯.

,발화자,발화문,QA번호,QA여부,감성,인텐트,상담번호,상담내순번
155756,s,네 4일자로 바꿔드릴게요.,875,a,m,예약_변경_요청,875,2
155755,c,며칠 전에 #이름#로 쿠키 픽업 예약 드렸었는데 혹시 4일날 픽업으로 바꿔주실 수 ...,875,q,m,예약_변경_요청,875,1
137976,c,A세트 구성에서는 카페라떼가 두 잔 나오는게 맞죠?,1630,q,m,제품_구성_확인,1630,1
137977,s,네 제일 많이 찾으시는 메뉴로 두 잔 나갑니다.,1630,a,m,제품_구성_확인,1630,2
137944,c,A세트 주문하려고 하는데 카페모카 나오는거 맞는거죠?,1631,q,m,제품_구성_확인,1631,1
137945,s,네 세트에는 카페모카로 준비해드리고 있습니다.,1631,a,m,제품_구성_확인,1631,2
138212,c,A세트 시키면 헤이즐넛라떼가 나오는거죠?,1632,q,m,제품_구성_확인,1632,1
138213,s,네 달달한 세트라서 그렇습니다.,1632,a,m,제품_구성_확인,1632,2
138120,c,A세트 주문 넣었는데 자바칩프라페로 들어간 거죠?,1633,q,m,제품_구성_확인,1633,1
138121,s,네 제가 직접 주문 확인 했습니다.,1633,a,m,제품_구성_확인,1633,2


In [44]:
## Q-A 데이터프레임에 저장하기

# QA여부 = q (질문)
df_q = df_new[(df_new['QA여부']=='q')]

# QA여부 = a (답변)
df_a = df_new[(df_new['QA여부']=='a')]

qlist = []
alist = []

for q, a in zip(df_q['발화문'], df_a['발화문']):
  qlist.append(q)
  alist.append(a)

df_qa = pd.DataFrame(columns = ['Q', 'A'])

df_qa['Q'] = qlist
df_qa['A'] = alist

df_qa.head()

,Q,A
0,며칠 전에 #이름#로 쿠키 픽업 예약 드렸었는데 혹시 4일날 픽업으로 바꿔주실 수 ...,네 4일자로 바꿔드릴게요.
1,A세트 구성에서는 카페라떼가 두 잔 나오는게 맞죠?,네 제일 많이 찾으시는 메뉴로 두 잔 나갑니다.
2,A세트 주문하려고 하는데 카페모카 나오는거 맞는거죠?,네 세트에는 카페모카로 준비해드리고 있습니다.
3,A세트 시키면 헤이즐넛라떼가 나오는거죠?,네 달달한 세트라서 그렇습니다.
4,A세트 주문 넣었는데 자바칩프라페로 들어간 거죠?,네 제가 직접 주문 확인 했습니다.


In [45]:
# 어긋나는 부분 있는지 눈으로 확인해보기 (1)

df_qa.tail(50)

,Q,A
56113,자바칩 프라푸치노에 휘핑크림 올려주실래요?,"네, 휘핑크림 올려 드리겠습니다."
56114,바닐라 라떼 우유는 저지방 우유로 주세요.,"네, 저지방 우유로 만들어 드릴게요."
56115,먹고 갈 건데 커피 나오는지 어떻게 알아요?,진동벨 울리면 진동벨을 가지고 카운터로 와 주시면 됩니다.
56116,어떤 게 지금 나온 빵이에요?,"좀 전에 호두스콘, 도넛이 막 구워져 나왔어요."
56117,크림치즈도넛 5개만 살게요.,"네, 계산해 드리겠습니다."
56118,저 베이글에 크림 치즈 발라주시겠어요?,"네, 크림 치즈 맛 고르시면 발라 드릴게요."
56119,선물용 카스테라 얼마예요?,그 카스테라는 13000원입니다.
56120,여기가 무한지대큐에 나왔던 빙수집인가요?,"네, 그 방송에 나온 가게 맞아요."
56121,오늘 가려고 하는데 가게 위치 좀 알려주세요.,이대역 3번 출구에 있습니다.
56122,여기 영업 시간은 어떻게 되나요?,오전 10시부터 오후 11시까지 영업합니다.


In [46]:
# 어긋나는 부분 있는지 눈으로 확인해보기 (2)

df_qa.head(50)

,Q,A
0,며칠 전에 #이름#로 쿠키 픽업 예약 드렸었는데 혹시 4일날 픽업으로 바꿔주실 수 ...,네 4일자로 바꿔드릴게요.
1,A세트 구성에서는 카페라떼가 두 잔 나오는게 맞죠?,네 제일 많이 찾으시는 메뉴로 두 잔 나갑니다.
2,A세트 주문하려고 하는데 카페모카 나오는거 맞는거죠?,네 세트에는 카페모카로 준비해드리고 있습니다.
3,A세트 시키면 헤이즐넛라떼가 나오는거죠?,네 달달한 세트라서 그렇습니다.
4,A세트 주문 넣었는데 자바칩프라페로 들어간 거죠?,네 제가 직접 주문 확인 했습니다.
5,혹시 아메리카노 몇 잔이나 나오는지 확인해주실 수 있나요?,네 세 잔정도 나올 것 같아요.
6,바닐라라떼 올라가는 토핑 재고가 아직 있는지 확인 부탁드려요.,네 토핑 재료는 조금 남았다고 하네요.
7,헤이즐넛라떼 열 잔 주문해야 되는 재고가 되나요?,네 재고는 되는 지 확인 해드리겠습니다.
8,자바칩프라페 열 잔 시키려고 하는데 재고 확인 가능할까요?,네 지금 주방에서 재고 확인해보고 연락 드릴게요.
9,이제 아메리카노 몇 잔 안나온다고 하셨어요.,확인 해보고 주문 처리해드릴게요.


확인 결과 다음 단계로 진행해도 될 것으로 판단.

In [47]:
# 추가 전처리 정규표현식 - 구두점(?.,!) 없애고 최종 학습 데이터 준비

questions = []

for sentence in df_qa['Q']:
    sentence = re.sub(r"([?.!,])", r" ", sentence)
    sentence = sentence.strip()
    questions.append(sentence)

answers = []

for sentence in df_qa['A']:
    sentence = re.sub(r"([?.!,])", r" ", sentence)
    sentence = sentence.strip()
    answers.append(sentence)

print(questions[:3])
print(answers[:3])

['며칠 전에 #이름#로 쿠키 픽업 예약 드렸었는데 혹시 4일날 픽업으로 바꿔주실 수 있나요', 'A세트 구성에서는 카페라떼가 두 잔 나오는게 맞죠', 'A세트 주문하려고 하는데 카페모카 나오는거 맞는거죠']
['네 4일자로 바꿔드릴게요', '네 제일 많이 찾으시는 메뉴로 두 잔 나갑니다', '네 세트에는 카페모카로 준비해드리고 있습니다']


df_qa 데이터 준비 완료.

### 단어집합(Vocabulary) 생성 - Subword TextEncoder
- 공식문서 [링크](https://www.tensorflow.org/datasets/api_docs/python/tfds/deprecated/text/SubwordTextEncoder)

In [61]:
# 질문, 답변 데이터로부터 단어 집합 생성
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    questions + answers, target_vocab_size=2**13)

In [ ]:
# 시작 토큰과 종료 토큰에 대한 정수 부여.
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]

# 시작 토큰과 종료 토큰을 고려하여 단어 집합의 크기를 + 2
VOCAB_SIZE = tokenizer.vocab_size + 2

print('시작 토큰 번호 :',START_TOKEN)
print('종료 토큰 번호 :',END_TOKEN)
print('단어 집합의 크기 :',VOCAB_SIZE)

시작 토큰 번호 : [8294]
종료 토큰 번호 : [8295]
단어 집합의 크기 : 8296


### 정수인코딩, 패딩

In [62]:
# 서브워드텍스트인코더 토크나이저의 .encode()를 사용하여 텍스트 시퀀스를 정수 시퀀스로 변환.
print('임의의 질문 샘플을 정수 인코딩 : {}'.format(tokenizer.encode(questions[5])))

임의의 질문 샘플을 정수 인코딩 : [38, 23, 40, 660, 890, 3004, 4699, 170, 3, 7]


In [65]:
# 서브워드텍스트인코더 토크나이저의 .encode()와 .decode() 테스트해보기
# 임의의 입력 문장을 sample_string에 저장
sample_string = questions[30]

# encode() : 텍스트 시퀀스 --> 정수 시퀀스
tokenized_string = tokenizer.encode(sample_string)
print ('정수 인코딩 후의 문장 {}'.format(tokenized_string))

# decode() : 정수 시퀀스 --> 텍스트 시퀀스
original_string = tokenizer.decode(tokenized_string)
print ('기존 문장: {}'.format(original_string))

정수 인코딩 후의 문장 [2436, 42, 4570, 679, 3827, 563, 201, 707, 5629]
기존 문장: 카페라떼를 집 냉장고에 두시간 이상 보관해도 되겠죠


In [ ]:
# 최대 길이를 30으로 정의 (임의 지정)
MAX_LENGTH = 30

# 토큰화 / 정수 인코딩 / 시작 토큰과 종료 토큰 추가 / 패딩
def tokenize_and_filter(inputs, outputs):
  tokenized_inputs, tokenized_outputs = [], []

  for (sentence1, sentence2) in zip(inputs, outputs):
    # encode(토큰화 + 정수 인코딩), 시작 토큰과 종료 토큰 추가
    sentence1 = START_TOKEN + tokenizer.encode(sentence1) + END_TOKEN
    sentence2 = START_TOKEN + tokenizer.encode(sentence2) + END_TOKEN

    tokenized_inputs.append(sentence1)
    tokenized_outputs.append(sentence2)

  # 패딩
  tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_inputs, maxlen=MAX_LENGTH, padding='post')
  tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_outputs, maxlen=MAX_LENGTH, padding='post')

  return tokenized_inputs, tokenized_outputs

questions, answers = tokenize_and_filter(questions, answers)

In [ ]:
print('질문 데이터의 크기(shape) :', questions.shape)
print('답변 데이터의 크기(shape) :', answers.shape)

질문 데이터의 크기(shape) : (56163, 30)
답변 데이터의 크기(shape) : (56163, 30)


In [ ]:
# 0번 샘플을 임의로 출력
print(questions[0])
print(answers[0])

[8294  853  392  691  375  302 8073   19   59  714   41 7101 1854   38
 2629 1576 7623 1572    3    7 8295    0    0    0    0    0    0    0
    0    0]
[8294   47 2629  388   19  652 8295    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]


# 모델링 및 평가
- 모델링 단계에서 [여기](https://https://wikidocs.net/31379)의 코드를 참조함.

## 트랜스포머 모델 구축


In [49]:
# 포지셔널 인코딩
class PositionalEncoding(tf.keras.layers.Layer):
  def __init__(self, position, d_model):
    super(PositionalEncoding, self).__init__()
    self.pos_encoding = self.positional_encoding(position, d_model)

  def get_angles(self, position, i, d_model):
    angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
    return position * angles

  def positional_encoding(self, position, d_model):
    angle_rads = self.get_angles(
        position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
        i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
        d_model=d_model)

    # 배열의 짝수 인덱스(2i)에는 사인 함수 적용
    sines = tf.math.sin(angle_rads[:, 0::2])

    # 배열의 홀수 인덱스(2i+1)에는 코사인 함수 적용
    cosines = tf.math.cos(angle_rads[:, 1::2])

    angle_rads = np.zeros(angle_rads.shape)
    angle_rads[:, 0::2] = sines
    angle_rads[:, 1::2] = cosines
    pos_encoding = tf.constant(angle_rads)
    pos_encoding = pos_encoding[tf.newaxis, ...]

    print(pos_encoding.shape)
    return tf.cast(pos_encoding, tf.float32)

  def call(self, inputs):
    return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

In [50]:
def scaled_dot_product_attention(query, key, value, mask):
  # query 크기 : (batch_size, num_heads, query의 문장 길이, d_model/num_heads)
  # key 크기 : (batch_size, num_heads, key의 문장 길이, d_model/num_heads)
  # value 크기 : (batch_size, num_heads, value의 문장 길이, d_model/num_heads)
  # padding_mask : (batch_size, 1, 1, key의 문장 길이)

  # Q와 K의 곱. 어텐션 스코어 행렬.
  matmul_qk = tf.matmul(query, key, transpose_b=True)

  # 스케일링
  # dk의 루트값으로 나눠준다.
  depth = tf.cast(tf.shape(key)[-1], tf.float32)
  logits = matmul_qk / tf.math.sqrt(depth)

  # 마스킹. 어텐션 스코어 행렬의 마스킹 할 위치에 매우 작은 음수값을 넣는다.
  # 매우 작은 값이므로 소프트맥스 함수를 지나면 행렬의 해당 위치의 값은 0이 된다.
  if mask is not None:
    logits += (mask * -1e9)

  # 소프트맥스 함수는 마지막 차원인 key의 문장 길이 방향으로 수행된다.
  # attention weight : (batch_size, num_heads, query의 문장 길이, key의 문장 길이)
  attention_weights = tf.nn.softmax(logits, axis=-1)

  # output : (batch_size, num_heads, query의 문장 길이, d_model/num_heads)
  output = tf.matmul(attention_weights, value)

  return output, attention_weights

In [51]:
class MultiHeadAttention(tf.keras.layers.Layer):

  def __init__(self, d_model, num_heads, name="multi_head_attention"):
    super(MultiHeadAttention, self).__init__(name=name)
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    # d_model을 num_heads로 나눈 값.
    # 논문 기준 : 64
    self.depth = d_model // self.num_heads

    # WQ, WK, WV에 해당하는 밀집층 정의
    self.query_dense = tf.keras.layers.Dense(units=d_model)
    self.key_dense = tf.keras.layers.Dense(units=d_model)
    self.value_dense = tf.keras.layers.Dense(units=d_model)

    # WO에 해당하는 밀집층 정의
    self.dense = tf.keras.layers.Dense(units=d_model)

  # num_heads 개수만큼 q, k, v를 split하는 함수
  def split_heads(self, inputs, batch_size):
    inputs = tf.reshape(
        inputs, shape=(batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(inputs, perm=[0, 2, 1, 3])

  def call(self, inputs):
    query, key, value, mask = inputs['query'], inputs['key'], inputs[
        'value'], inputs['mask']
    batch_size = tf.shape(query)[0]

    # 1. WQ, WK, WV에 해당하는 밀집층 지나기
    # q : (batch_size, query의 문장 길이, d_model)
    # k : (batch_size, key의 문장 길이, d_model)
    # v : (batch_size, value의 문장 길이, d_model)
    # 참고) 인코더(k, v)-디코더(q) 어텐션에서는 query 길이와 key, value의 길이는 다를 수 있다.
    query = self.query_dense(query)
    key = self.key_dense(key)
    value = self.value_dense(value)

    # 2. 헤드 나누기
    # q : (batch_size, num_heads, query의 문장 길이, d_model/num_heads)
    # k : (batch_size, num_heads, key의 문장 길이, d_model/num_heads)
    # v : (batch_size, num_heads, value의 문장 길이, d_model/num_heads)
    query = self.split_heads(query, batch_size)
    key = self.split_heads(key, batch_size)
    value = self.split_heads(value, batch_size)

    # 3. 스케일드 닷 프로덕트 어텐션. 앞서 구현한 함수 사용.
    # (batch_size, num_heads, query의 문장 길이, d_model/num_heads)
    scaled_attention, _ = scaled_dot_product_attention(query, key, value, mask)
    # (batch_size, query의 문장 길이, num_heads, d_model/num_heads)
    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

    # 4. 헤드 연결(concatenate)하기
    # (batch_size, query의 문장 길이, d_model)
    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))

    # 5. WO에 해당하는 밀집층 지나기
    # (batch_size, query의 문장 길이, d_model)
    outputs = self.dense(concat_attention)

    return outputs

In [52]:
def create_padding_mask(x):
  mask = tf.cast(tf.math.equal(x, 0), tf.float32)
  # (batch_size, 1, 1, key의 문장 길이)
  return mask[:, tf.newaxis, tf.newaxis, :]

In [53]:
def encoder_layer(dff, d_model, num_heads, dropout, name="encoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")

  # 인코더는 패딩 마스크 사용
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  # 멀티-헤드 어텐션 (첫번째 서브층 / 셀프 어텐션)
  attention = MultiHeadAttention(
      d_model, num_heads, name="attention")({
          'query': inputs, 'key': inputs, 'value': inputs, # Q = K = V
          'mask': padding_mask # 패딩 마스크 사용
      })

  # 드롭아웃 + 잔차 연결과 층 정규화
  attention = tf.keras.layers.Dropout(rate=dropout)(attention)
  attention = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(inputs + attention)

  # 포지션 와이즈 피드 포워드 신경망 (두번째 서브층)
  outputs = tf.keras.layers.Dense(units=dff, activation='relu')(attention)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)

  # 드롭아웃 + 잔차 연결과 층 정규화
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention + outputs)

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [54]:
def encoder(vocab_size, num_layers, dff,
            d_model, num_heads, dropout,
            name="encoder"):
  inputs = tf.keras.Input(shape=(None,), name="inputs")

  # 인코더는 패딩 마스크 사용
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  # 포지셔널 인코딩 + 드롭아웃
  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)
  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  # 인코더를 num_layers개 쌓기
  for i in range(num_layers):
    outputs = encoder_layer(dff=dff, d_model=d_model, num_heads=num_heads,
        dropout=dropout, name="encoder_layer_{}".format(i),
    )([outputs, padding_mask])

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [55]:
# 디코더의 첫번째 서브층(sublayer)에서 미래 토큰을 Mask하는 함수
def create_look_ahead_mask(x):
  seq_len = tf.shape(x)[1]
  look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
  padding_mask = create_padding_mask(x) # 패딩 마스크도 포함
  return tf.maximum(look_ahead_mask, padding_mask)

In [56]:
def decoder_layer(dff, d_model, num_heads, dropout, name="decoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
  enc_outputs = tf.keras.Input(shape=(None, d_model), name="encoder_outputs")

  # 룩어헤드 마스크(첫번째 서브층)
  look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name="look_ahead_mask")

  # 패딩 마스크(두번째 서브층)
  padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

  # 멀티-헤드 어텐션 (첫번째 서브층 / 마스크드 셀프 어텐션)
  attention1 = MultiHeadAttention(
      d_model, num_heads, name="attention_1")(inputs={
          'query': inputs, 'key': inputs, 'value': inputs, # Q = K = V
          'mask': look_ahead_mask # 룩어헤드 마스크
      })

  # 잔차 연결과 층 정규화
  attention1 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention1 + inputs)

  # 멀티-헤드 어텐션 (두번째 서브층 / 디코더-인코더 어텐션)
  attention2 = MultiHeadAttention(
      d_model, num_heads, name="attention_2")(inputs={
          'query': attention1, 'key': enc_outputs, 'value': enc_outputs, # Q != K = V
          'mask': padding_mask # 패딩 마스크
      })

  # 드롭아웃 + 잔차 연결과 층 정규화
  attention2 = tf.keras.layers.Dropout(rate=dropout)(attention2)
  attention2 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention2 + attention1)

  # 포지션 와이즈 피드 포워드 신경망 (세번째 서브층)
  outputs = tf.keras.layers.Dense(units=dff, activation='relu')(attention2)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)

  # 드롭아웃 + 잔차 연결과 층 정규화
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(outputs + attention2)

  return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

In [57]:
def decoder(vocab_size, num_layers, dff,
            d_model, num_heads, dropout,
            name='decoder'):
  inputs = tf.keras.Input(shape=(None,), name='inputs')
  enc_outputs = tf.keras.Input(shape=(None, d_model), name='encoder_outputs')

  # 디코더는 룩어헤드 마스크(첫번째 서브층)와 패딩 마스크(두번째 서브층) 둘 다 사용.
  look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name='look_ahead_mask')
  padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

  # 포지셔널 인코딩 + 드롭아웃
  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)
  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  # 디코더를 num_layers개 쌓기
  for i in range(num_layers):
    outputs = decoder_layer(dff=dff, d_model=d_model, num_heads=num_heads,
        dropout=dropout, name='decoder_layer_{}'.format(i),
    )(inputs=[outputs, enc_outputs, look_ahead_mask, padding_mask])

  return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

In [58]:
def transformer(vocab_size, num_layers, dff,
                d_model, num_heads, dropout,
                name="transformer"):

  # 인코더의 입력
  inputs = tf.keras.Input(shape=(None,), name="inputs")

  # 디코더의 입력
  dec_inputs = tf.keras.Input(shape=(None,), name="dec_inputs")

  # 인코더의 패딩 마스크
  enc_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='enc_padding_mask')(inputs)

  # 디코더의 룩어헤드 마스크(첫번째 서브층)
  look_ahead_mask = tf.keras.layers.Lambda(
      create_look_ahead_mask, output_shape=(1, None, None),
      name='look_ahead_mask')(dec_inputs)

  # 디코더의 패딩 마스크(두번째 서브층)
  dec_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='dec_padding_mask')(inputs)

  # 인코더의 출력은 enc_outputs. 디코더로 전달된다.
  enc_outputs = encoder(vocab_size=vocab_size, num_layers=num_layers, dff=dff,
      d_model=d_model, num_heads=num_heads, dropout=dropout,
  )(inputs=[inputs, enc_padding_mask]) # 인코더의 입력은 입력 문장과 패딩 마스크

  # 디코더의 출력은 dec_outputs. 출력층으로 전달된다.
  dec_outputs = decoder(vocab_size=vocab_size, num_layers=num_layers, dff=dff,
      d_model=d_model, num_heads=num_heads, dropout=dropout,
  )(inputs=[dec_inputs, enc_outputs, look_ahead_mask, dec_padding_mask])

  # 다음 단어 예측을 위한 출력층
  outputs = tf.keras.layers.Dense(units=vocab_size, name="outputs")(dec_outputs)

  return tf.keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)

In [59]:
def loss_function(y_true, y_pred):
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))

  loss = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction='none')(y_true, y_pred)

  mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
  loss = tf.multiply(loss, mask)

  return tf.reduce_mean(loss)


In [60]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):

  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()
    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)
    self.warmup_steps = warmup_steps

  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps**-1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

## 모델링

In [ ]:
# 인코더, 디코더 입력 + 레이블 만들기

BATCH_SIZE = 64
BUFFER_SIZE = 20000

# 디코더의 실제값 시퀀스에서는 시작 토큰을 제거해야 한다.
dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': questions,
        'dec_inputs': answers[:, :-1] # 디코더의 입력. 마지막 패딩 토큰이 제거된다.
    },
    {
        'outputs': answers[:, 1:]  # 맨 처음 토큰이 제거된다. 다시 말해 시작 토큰이 제거된다.
    },
))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
tf.keras.backend.clear_session()

# 하이퍼파라미터
D_MODEL = 256
NUM_LAYERS = 2
NUM_HEADS = 8
DFF = 512
DROPOUT = 0.1

model = transformer(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS,
    dff=DFF,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT)

(1, 8296, 256)
(1, 8296, 256)


In [ ]:
learning_rate = CustomSchedule(D_MODEL)

optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y_true, y_pred):
  # 레이블의 크기는 (batch_size, MAX_LENGTH - 1)
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
  return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])

In [ ]:
# 모델 학습
EPOCHS = 30
model.fit(dataset, epochs=EPOCHS)

Epoch 1/30
878/878 [==============================] - 51s 48ms/step - loss: 1.7716 - accuracy: 0.0674
Epoch 2/30
878/878 [==============================] - 42s 48ms/step - loss: 1.2743 - accuracy: 0.0959
Epoch 3/30
878/878 [==============================] - 43s 49ms/step - loss: 1.0017 - accuracy: 0.1149
Epoch 4/30
878/878 [==============================] - 43s 49ms/step - loss: 0.8301 - accuracy: 0.1281
Epoch 5/30
878/878 [==============================] - 42s 48ms/step - loss: 0.7233 - accuracy: 0.1379
Epoch 6/30
878/878 [==============================] - 43s 49ms/step - loss: 0.6219 - accuracy: 0.1495
Epoch 7/30
878/878 [==============================] - 43s 49ms/step - loss: 0.5319 - accuracy: 0.1609
Epoch 8/30
878/878 [==============================] - 42s 48ms/step - loss: 0.4602 - accuracy: 0.1718
Epoch 9/30
878/878 [==============================] - 42s 48ms/step - loss: 0.4009 - accuracy: 0.1812
Epoch 10/30
878/878 [==============================] - 43s 49ms/step - loss: 0.351

## 모델 평가

In [ ]:
# 구두점 제거
def preprocess_sentence(sentence):
  sentence = re.sub(r"([?.!,])", r" ", sentence)
  sentence = sentence.strip()
  return sentence

# 출력값 벡터
def evaluate(sentence):
  # 입력 문장에 대한 전처리
  sentence = preprocess_sentence(sentence)

  # 입력 문장에 시작 토큰과 종료 토큰을 추가
  sentence = tf.expand_dims(
      START_TOKEN + tokenizer.encode(sentence) + END_TOKEN, axis=0)

  output = tf.expand_dims(START_TOKEN, 0)

  # 디코더의 예측 시작
  for i in range(MAX_LENGTH):
    predictions = model(inputs=[sentence, output], training=False)

    # 현재 시점의 예측 단어를 받아온다.
    predictions = predictions[:, -1:, :]
    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

    # 만약 현재 시점의 예측 단어가 종료 토큰이라면 예측을 중단
    if tf.equal(predicted_id, END_TOKEN[0]):
      break

    # 현재 시점의 예측 단어를 output(출력)에 연결한다.
    # output은 for문의 다음 루프에서 디코더의 입력이 된다.
    output = tf.concat([output, predicted_id], axis=-1)

  # 단어 예측이 모두 끝났다면 output을 리턴.
  return tf.squeeze(output, axis=0)

# 최종 출력 
def predict(sentence):
  prediction = evaluate(sentence)

  # prediction == 디코더가 리턴한 챗봇의 대답에 해당하는 정수 시퀀스
  # tokenizer.decode()를 통해 정수 시퀀스를 문자열로 디코딩.
  predicted_sentence = tokenizer.decode(
      [i for i in prediction if i < tokenizer.vocab_size])

  print('Input: {}'.format(sentence))
  print('Output: {}'.format(predicted_sentence))

  return predicted_sentence

In [ ]:
output = predict('배달 되나요?')

Input: 배달 되나요?
Output: 네  배달 됩니다


In [ ]:
output = predict('아메리카노 디카페인 있나요?')

Input: 아메리카노 디카페인 있나요?
Output: 네  있습니다


In [ ]:
output = predict('아메리카노 얼마에요?')

Input: 아메리카노 얼마에요?
Output: 5000원입니다


In [ ]:
output = predict('차 종류는 뭐가 있어요?')

Input: 차 종류는 뭐가 있어요?
Output: 현재 판매 중인 인절미에는 딸기인절미  카스테라인절미  흑임자인절미  클래식인절미꿀떡이 있습니다


In [ ]:
output = predict('여기 뭐가 맛있어요?')

Input: 여기 뭐가 맛있어요?
Output: 자몽젤라또가 맛있습니다


와 대답 잘 뽑아내네. 이외에도 다양하게 체크해봄.

In [ ]:
# 이런 건 좀 어색해보이기도 하네.
output = predict('3명 포장 안 되나요?')

Input: 3명 포장 안 되나요?
Output: 매장 해드릴게요
